In [117]:
import pandas as pd

In [118]:
import numpy as np

In [119]:
import streamlit as st

In [120]:
import matplotlib.pyplot as plt

In [121]:
from sklearn.linear_model import LogisticRegression

In [122]:
from sklearn.model_selection import train_test_split

In [123]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [124]:
df = pd.read_csv("retail_cleans.csv")

In [125]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,Year,Month,MonthYear
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,2009,dic,2009-12-01
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,dic,2009-12-01
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,2009,dic,2009-12-01
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,2009,dic,2009-12-01
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,2009,dic,2009-12-01


In [126]:
df.columns


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'TotalPrice', 'Year', 'Month',
       'MonthYear'],
      dtype='object')

In [127]:
df.columns = df.columns.str.strip()


In [128]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [129]:
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

In [130]:
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "Invoice": "nunique",
    "TotalPrice": "sum"
})

In [131]:
print(df.columns)


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'TotalPrice', 'Year', 'Month',
       'MonthYear'],
      dtype='object')


In [132]:
rfm.columns = ["Recency", "Frequency", "Monetary"]
rfm.reset_index(inplace=True)

In [133]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary
0,12346,165,11,372.86
1,12347,3,2,1323.32
2,12348,74,1,222.16
3,12349,43,3,2671.14
4,12351,11,1,300.93


df.columns


In [134]:
rfm.shape

(4312, 4)

In [135]:
rfm["Churn"] = np.where(rfm["Recency"] > 90, 1, 0)

In [136]:
rfm["Churn"].mean()

np.float64(0.3327922077922078)

In [137]:
X = rfm[["Frequency", "Monetary"]]
y = rfm["Churn"]

In [138]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [139]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [140]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

In [141]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

Accuracy: 0.7219003476245655
ROC-AUC: 0.7736787179425665


In [142]:
rfm["Churn_Prob"] = model.predict_proba(X)[:,1]

In [143]:
rfm["CLV"] = rfm["Monetary"] * rfm["Frequency"]

In [144]:
clv_threshold = rfm["CLV"].median()

In [145]:
risk_threshold = rfm["Churn_Prob"].median()

In [146]:
def segment(row):
    if row["CLV"] >= clv_threshold and row["Churn_Prob"] >= risk_threshold:
        return "High Value - High Risk"
    elif row["CLV"] >= clv_threshold and row["Churn_Prob"] < risk_threshold:
        return "High Value - Low Risk"
    elif row["CLV"] < clv_threshold and row["Churn_Prob"] >= risk_threshold:
        return "Low Value - High Risk"
    else:
        return "Low Value - Low Risk"

rfm["Segment"] = rfm.apply(segment, axis=1)

In [147]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Churn,Churn_Prob,CLV,Segment
0,12346,165,11,372.86,1,0.024160,4101.46,High Value - Low Risk
1,12347,3,2,1323.32,0,0.411732,2646.64,High Value - High Risk
2,12348,74,1,222.16,0,0.510352,222.16,Low Value - High Risk
3,12349,43,3,2671.14,0,0.318530,8013.42,High Value - Low Risk
4,12351,11,1,300.93,0,0.509913,300.93,Low Value - High Risk


In [148]:
rfm["Segment"].value_counts()


Segment
High Value - Low Risk     2009
Low Value - High Risk     2009
High Value - High Risk     147
Low Value - Low Risk       147
Name: count, dtype: int64

In [149]:
print("Total Customers:", len(rfm))

Total Customers: 4312


In [150]:
print("Churn Rate:", round(rfm["Churn"].mean()*100,2), "%")

Churn Rate: 33.28 %


In [152]:
print("High Value High Risk Customers:", 
      len(rfm[rfm["Segment"]=="High Value - High Risk"]))

High Value High Risk Customers: 147


In [153]:
at_risk_revenue = rfm[rfm["Segment"]=="High Value - High Risk"]["CLV"].sum()

print("Revenue At Risk:", at_risk_revenue)



Revenue At Risk: 342198.46199999994


In [154]:
rfm.to_csv("rfm_output.csv", index=False)
